In [1]:
import pandas as pd
import numpy as np

In [5]:
# import training data
dft_eng = pd.read_csv('../data/dft_eng.csv')
dft_jap = pd.read_csv('../data/dft_jap.csv')
dft_fin = pd.read_csv('../data/dft_fin.csv')

# import validation data
dfv_eng = pd.read_csv('../data/dfv_eng.csv')
dfv_jap = pd.read_csv('../data/dfv_jap.csv')
dfv_fin = pd.read_csv('../data/dfv_fin.csv')

In [6]:
dft_eng

,Unnamed: 0,question_text,document_title,language,annotations,document_plaintext,document_url,question_text_tokenized,document_plaintext_tokenized
0,0,When was quantum field theory developed?,Quantum field theory,english,"{'answer_start': array([159]), 'answer_text': ...",Quantum field theory naturally began with the ...,https://en.wikipedia.org/wiki/Quantum%20field%...,"['When', 'was', 'quantum', 'field', 'theory', ...","['Quantum', 'field', 'theory', 'naturally', 'b..."
1,1,Who was the first Nobel prize winner for Liter...,List of Nobel laureates in Literature,english,"{'answer_start': array([610]), 'answer_text': ...",The Nobel Prize in Literature (Swedish: Nobelp...,https://en.wikipedia.org/wiki/List%20of%20Nobe...,"['Who', 'was', 'the', 'first', 'Nobel', 'prize...","['The', 'Nobel', 'Prize', 'in', 'Literature', ..."
2,2,When is the dialectical method used?,Dialectic,english,"{'answer_start': array([129]), 'answer_text': ...","Dialectic or dialectics (Greek: διαλεκτική, di...",https://en.wikipedia.org/wiki/Dialectic,"['When', 'is', 'the', 'dialectical', 'method',...","['Dialectic', 'or', 'dialectics', '(', 'Greek'..."
3,3,Who invented Hangul?,Origin of Hangul,english,"{'answer_start': array([88]), 'answer_text': a...",Hangul was personally created and promulgated ...,https://en.wikipedia.org/wiki/Origin%20of%20Ha...,"['Who', 'invented', 'Hangul', '?']","['Hangul', 'was', 'personally', 'created', 'an..."
4,4,What do Grasshoppers eat?,Grasshopper,english,"{'answer_start': array([0]), 'answer_text': ar...","Grasshoppers are plant-eaters, with a few spec...",https://en.wikipedia.org/wiki/Grasshopper,"['What', 'do', 'Grasshoppers', 'eat', '?']","['Grasshoppers', 'are', 'plant-eaters', ',', '..."
...,...,...,...,...,...,...,...,...,...
7384,7384,What was Neil Brooks' fastest recorded time?,Swimming at the 1980 Summer Olympics – Men's 4...,english,"{'answer_start': array([-1]), 'answer_text': a...",The medley relay was scheduled in the Olympisk...,https://en.wikipedia.org/wiki/Swimming%20at%20...,"['What', 'was', 'Neil', 'Brooks', ""'"", 'fastes...","['The', 'medley', 'relay', 'was', 'scheduled',..."
7385,7385,Who are the three most important eastern philo...,Eastern philosophy,english,"{'answer_start': array([-1]), 'answer_text': a...",Sāmkhya is a dualist philosophical tradition b...,https://en.wikipedia.org/wiki/Eastern%20philos...,"['Who', 'are', 'the', 'three', 'most', 'import...","['Sāmkhya', 'is', 'a', 'dualist', 'philosophic..."
7386,7386,Who was costume designer for the first Star Wa...,John Mollo,english,"{'answer_start': array([-1]), 'answer_text': a...",Mollo was surprised by the success of Star War...,https://en.wikipedia.org/wiki/John%20Mollo,"['Who', 'was', 'costume', 'designer', 'for', '...","['Mollo', 'was', 'surprised', 'by', 'the', 'su..."
7387,7387,Who developed the first thermonuclear weapon?,History of nuclear weapons,english,"{'answer_start': array([-1]), 'answer_text': a...","In the end, President Truman made the final de...",https://en.wikipedia.org/wiki/History%20of%20n...,"['Who', 'developed', 'the', 'first', 'thermonu...","['In', 'the', 'end', ',', 'President', 'Truman..."
